In [2]:
from scipy.sparse.linalg import bicgstab
import numpy as np
from numpy import linalg as lg
from scipy.io import mmread
from tabulate import tabulate
from time import time

A1=np.array(mmread('/content/bcsstk01.mtx.gz').todense())
#A1.shape=(48,46) Non symmetrique
A2=np.array(mmread('/content/impcol_c.mtx.gz').todense())
#A2.shape=(137,137) Non symmetrique
A3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
#A3.shape=(207,207) Non symmetrique
A4=np.array(mmread('/content/mcca.mtx.gz').todense())
#A4.shape=(180, 180) Non symmetrique
A5=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
#A5.shape=(3562,3562) Symmetrique definie positive
A6=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
#A6.shape=(817,817) Symmetrique definie positive

B1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())             
# B1.shape=(1806, 1806) real symmetric positive definite
B3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# B3.shape=(485,485) real symmetric positive definite
B4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
B5=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B6=np.array(mmread('/content/bcsstk16.mtx.gz').todense())
B7=np.array(mmread('/content/bcsstk17.mtx.gz').todense())
B8=np.array(mmread('/content/bcsstk18.mtx.gz').todense())
B9=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
B10=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
B11=np.array(mmread('/content/bcsstm26.mtx.gz').todense())

C1=np.array(mmread('/content/west0067.mtx.gz').todense())              
C2=np.array(mmread('/content/jpwh_991.mtx.gz').todense())
C3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
C4=np.array(mmread('/content/impcol_e.mtx.gz').todense())
C5=np.array(mmread('/content/sherman4.mtx.gz').todense())
C6=np.array(mmread('/content/orsreg_1.mtx.gz').todense())
C7=np.array(mmread('/content/orsirr_1.mtx.gz').todense())
C8=np.array(mmread('/content/add20.mtx.gz').todense())
C9=np.array(mmread('/content/gre__185.mtx.gz').todense())
C10=np.array(mmread('/content/fs_680_3.mtx.gz').todense())
C11=np.array(mmread('/content/fs_680_2.mtx.gz').todense())
C12=np.array(mmread('/content/west0381.mtx.gz').todense())
C13=np.array(mmread('/content/e05r0200.mtx.gz').todense())

#**BICGSTAB  Method**





In [3]:

def BICGSTAB(A,b,x0,tol,maxiter):
  r0=b-np.dot(A,x0)
  rt=r0/(lg.norm(r0)*lg.norm(r0))
  
  p0=r0
  for i in range(maxiter):
    alpha=np.dot(r0,rt)/np.dot(np.dot(A,p0),rt)
    s0=r0-alpha*np.dot(A,p0)
    w0=np.dot(np.dot(A,s0),s0)/np.dot(np.dot(A,s0),np.dot(A,s0))
    x0=x0+alpha*p0+np.dot(w0,s0)
    r1=s0-np.dot(w0,np.dot(A,s0))
    beta=(np.dot(r1,rt)/np.dot(r0,rt))*(alpha/w0)
    p0=r1+beta*(p0-np.dot(w0,np.dot(A,p0)))
    r0=r1
    if lg.norm(r1)<tol:
      break
  return x0,i+1

def bicgstab_iter(A, b,x0,tol,maxiter):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return bicgstab(A, b, callback=callback,tol = tol,maxiter=maxiter,x0=x0)[0],num_iters

# **Cluster 1:**

In [3]:
matricies=[B5,C1,B9,C2]
tol=1e-5
itermax = 10000

In [4]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicgstab_iter(matricies[i],b,x,tol,itermax)[1]
  t3 = time()
  y=bicgstab(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICGSTAB(matricies[i], b, x, tol,itermax)  
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-----------+---------------+------------------+-----------+---------------+
| matricies    |   condition number |   Our_nbr_iter |      time |           res |   scipy_nbr_iter |   sc_time |        sc_res |
+==============+====================+================+===========+===============+==================+===========+===============+
| (3600, 3600) |            23.7154 |              3 | 183.42    |   1.77161e-11 |                3 |  49.7808  |   7.22899e-12 |
+--------------+--------------------+----------------+-----------+---------------+------------------+-----------+---------------+
| (67, 67)     |           130.217  |          10000 | 626.285   | 134.089       |               83 |   7.71379 | 156.315       |
+--------------+--------------------+----------------+-----------+---------------+------------------+-----------+---------------+
| (138, 138)   |           941.295  |             47 |   5.37276 |   3.72359e-06 |        

# **Cluster 2:**

In [5]:
matricies=[C7,B10,A6,B11,C8,C9]
tol=1e-5
itermax = 10000

In [6]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicgstab_iter(matricies[i],b,x,tol,itermax)[1]
  t3 = time()
  y=bicgstab(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICGSTAB(matricies[i], b, x, tol,itermax)  
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+----------+-------------+------------------+-----------+--------------+
| matricies    |   condition number |   Our_nbr_iter |     time |         res |   scipy_nbr_iter |   sc_time |       sc_res |
+==============+====================+================+==========+=============+==================+===========+==============+
| (1030, 1030) |            77142.8 |           1566 |  4486.05 | 9.89553e-06 |             1027 |   995.387 |  0.000245136 |
+--------------+--------------------+----------------+----------+-------------+------------------+-----------+--------------+
| (1224, 1224) |            24135.9 |            747 |  3028.9  | 5.2391e-06  |              498 |   592.584 |  0.00033386  |
+--------------+--------------------+----------------+----------+-------------+------------------+-----------+--------------+
| (817, 817)   |           233734   |           1451 |  2285.35 | 4.21378e-06 |              973 |   592.677 |  0.0002

# **Cluster 3:**

In [6]:
matricies=[B6,B1,B4]
tol=1e-5
itermax = 100000

In [7]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicgstab_iter(matricies[i],b,x,tol,itermax)[1]
  t3 = time()
  y=bicgstab(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICGSTAB(matricies[i], b, x, tol,itermax)  
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------------+---------------+------------------+----------------+---------------+
| matricies    |   condition number |   Our_nbr_iter |             time |           res |   scipy_nbr_iter |        sc_time |        sc_res |
+==============+====================+================+==================+===============+==================+================+===============+
| (4884, 4884) |        4.94318e+09 |            422 |  46223.7         |   5.603e-06   |              359 | 11280.7        |   0.000695086 |
+--------------+--------------------+----------------+------------------+---------------+------------------+----------------+---------------+
| (1806, 1806) |        1.19232e+10 |          29802 | 312255           |   7.08835e-06 |            23287 | 73343.9        |   0.000214067 |
+--------------+--------------------+----------------+------------------+---------------+------------------+----------------+---------------+
| (356

# **Cluster 4:**

In [4]:
matricies=[C3,C4,C10,C12]
tol=1e-5
itermax = 10000

In [5]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicgstab_iter(matricies[i],b,x,tol,itermax)[1]
  t3 = time()
  y=bicgstab(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICGSTAB(matricies[i], b, x, tol,itermax)  
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+-------------+--------------------+----------------+----------+-----------------+------------------+-----------+------------------+
| matricies   |   condition number |   Our_nbr_iter |     time |             res |   scipy_nbr_iter |   sc_time |           sc_res |
+=============+====================+================+==========+=================+==================+===========+==================+
| (207, 207)  |        1.35164e+08 |          10000 |  1885.24 |     2.48564e+07 |              875 |   98.6507 | 117974           |
+-------------+--------------------+----------------+----------+-----------------+------------------+-----------+------------------+
| (225, 225)  |        7.10097e+06 |          10000 |  1749.89 |  2669.47        |             1604 |  182.433  |   9315.37        |
+-------------+--------------------+----------------+----------+-----------------+------------------+-----------+------------------+
| (680, 680)  |        3.58695e+06 |          10000 | 11606.2  |     